In [1]:
import chromadb
from chromadb.api.types import Embeddings, Metadata
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.config import Settings

In [2]:
client = chromadb.Client(
    Settings(chroma_db_impl="postgres", postgres_username="postgres", 
                                postgres_password="postgres", 
                                postgres_hostname="localhost", 
                                postgres_port="5445", 
                                postgres_databasename="postgres")
    )

# Everything that works with the client below should work with the above client
# client = chromadb.Client()

In [3]:
collection = client.create_collection("all-my-documents", get_or_create=True)
collection2 = client.create_collection("empty-collection", get_or_create=True)
collection3 = client.get_or_create_collection("extra-documents")

In [4]:
# list all collections
print(client.list_collections())
# get an existing collection
print(client.get_collection("all-my-documents"))

[Collection(name=extra-documents), Collection(name=all-my-documents), Collection(name=empty-collection)]
name='all-my-documents' id=UUID('2a48e5df-f275-4915-b97f-82805e8c2536') metadata=None


In [5]:
print(collection.count())
print(collection2.count())

2
0


In [6]:
# delete a collection that doesn't exist
try:
    client.delete_collection("dne-connection")
except ValueError:
    print("Collection doesn't exist and throws exception as expected")

Collection doesn't exist and throws exception as expected


In [9]:
# delete a collection that exists
client.delete_collection("empty-collection")

ValueError: Collection empty-collection does not exist

In [10]:
# collection.modify(name="updated-documents", metadata={"test": "metadata-test-success"})

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [7]:
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["This is document1", "This is document2"], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
    ids=["doc1", "doc2"], # unique for each doc
)

IDAlreadyExistsError: IDs ['doc1', 'doc2'] already exist in collection 2a48e5df-f275-4915-b97f-82805e8c2536

In [4]:
collection.count()

2

In [6]:
# upsert items. new items will be added, existing items will be updated.
collection.upsert(
    ids=["doc1", "doc2", "doc3"],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    documents=["doc1", "doc2", "doc3"],
)

IDAlreadyExistsError: IDs ['doc1', 'doc2'] already exist in collection 2a48e5df-f275-4915-b97f-82805e8c2536

In [4]:
collection.query(
    query_texts=["doc10", "thus spake zarathustra"], 
    n_results=10
    #where={"metadata_field": "is_equal_to_this"},
    #where_document={"$contains":"search_string"}
    )

[('2a48e5df-f275-4915-b97f-82805e8c2536', '3a70a6c2-a33e-47c8-b6fd-0318b7d86edf', '[-0.079880714,0.089616835,0.033762287,0.026681308,0.058535565,-0.011690716,0.008396599,0.023979163,0.009647535,-0.005084895,0.0010960803,0.08359419,-0.0005668837,-0.03032098,-0.08796833,0.024921473,-0.041935537,-0.03401016,0.00011679966,0.074820645,0.05398246,0.115735106,0.0062643783,-0.033620644,0.052878756,0.043765463,-0.099466264,0.028168574,0.039464276,-0.08018195,0.05720446,0.03847993,0.030348057,0.018265044,0.07134868,-0.0043168003,0.067193426,7.0845526e-05,-0.0028624372,0.06793803,0.016321283,-0.08489549,-0.05061706,-0.024112187,0.008140208,0.031960197,-0.0639242,0.038530238,-0.005406315,0.045207668,-0.04911522,-0.05539698,-0.10752249,0.0063627316,0.059205785,0.025650987,-0.049948655,0.06886511,-0.022425245,0.015455835,0.006478292,0.029562294,-0.061302174,0.032796662,0.10894897,0.009696123,0.016009642,-0.07203673,-0.10457947,-0.0041873166,-0.065601766,0.014561871,0.031771827,-0.04817276,-0.0086928

NotImplementedError: 